In [12]:
import queue 

q = queue.Queue()

q.put(1)
q.put(2)
q.put(3)

print(q.empty())
print(q.get())
print(q.get())
print(q.get())
print(q.empty())

False
1
2
3
True


In [25]:
q = queue.Queue(maxsize=10)

for i in range(5):
    q.put(i,block=False)
    print(f"put {i}")

    
while True:
# while q.qsize()>0:
    print(q.get(block=False))

put 0
put 1
put 2
put 3
put 4
0
1
2
3
4


Empty: 

In [1]:
### ref: https://shengyu7697.github.io/python-queue/
### Producer / Consumer with Queue !!!
import queue , time ,random , threading

q = queue.Queue(10)

class Producer(threading.Thread):
    def __init__(self,thread_name):
        super(Producer,self).__init__(name=thread_name)
        
    def run(self):
        global q 
        count = 1
        while True:
            if q.full():
                print("queue is full")
                pass
            
            msg = str(count)
            q.put(msg)
            print(f'{self.name} put {msg} , qsize : {str(q.qsize())}\n')
            
            count = count +1
            time.sleep(random.random()*2)
        
class Consumer(threading.Thread):
    def __init__(self,thread_name):
        super(Consumer,self).__init__(name=thread_name)
        
    def run(self):
        global q
        while True:
            if q.empty():
                print("queue is empty")
                pass
            
            msg = q.get()
            print(f'{self.name} get {msg} , qsize : {str(q.qsize())}\n')
            time.sleep(random.random()*3)
        
        
p = Producer("producer")
p.start()

c = Consumer("consumer")
c.start()


producer put 1 , qsize : 1
consumer get 1 , qsize : 0


producer put 2 , qsize : 1

producer put 3 , qsize : 2

consumer get 2 , qsize : 1

consumer get 3 , qsize : 0

producer put 4 , qsize : 1

producer put 5 , qsize : 2

consumer get 4 , qsize : 1

producer put 6 , qsize : 2

consumer get 5 , qsize : 1

producer put 7 , qsize : 2

consumer get 6 , qsize : 1

producer put 8 , qsize : 2

consumer get 7 , qsize : 1

producer put 9 , qsize : 2

producer put 10 , qsize : 3

producer put 11 , qsize : 4

consumer get 8 , qsize : 3

consumer get 9 , qsize : 2

producer put 12 , qsize : 3

producer put 13 , qsize : 4

producer put 14 , qsize : 5

consumer get 10 , qsize : 4

producer put 15 , qsize : 5

producer put 16 , qsize : 6

consumer get 11 , qsize : 5

producer put 17 , qsize : 6

consumer get 12 , qsize : 5

consumer get 13 , qsize : 4

producer put 18 , qsize : 5

consumer get 14 , qsize : 4

consumer get 15 , qsize : 3

producer put 19 , qsize : 4

producer put 20 , qsize : 5

pro

In [2]:
# ref: https://blog.csdn.net/brucewong0516/article/details/84587522
# Producer / Consumer with Condition !!!
import threading , time

class Producer(threading.Thread):
    def run(self):
        global cnt
        while True:
            ### 此為 critical section 啦, 有 mutex 的人才可執行！
            ### con.acquire() 取得某一個有 mutex 的 thread , 其餘只能繼續 waiting 
            if con.acquire():
                if cnt >900:
                    con.wait()
                else:
                    cnt += 100
                    msg = f"{self.name} produce 100, count ={str(cnt)}"
                    print(msg)
                    
                    # 從 waiting 池挑一個 thread 出來
                    # 通知其用 acquire 方法, 嘗試取得 mutex
                    con.notify() 
                    
                con.release()
                time.sleep(1)
            ###
                
class Consumer(threading.Thread):
    def run(self):
        global cnt
        while True:
            if con.acquire():
                if cnt < 100:
                    con.wait()
                else:
                    cnt -= 30
                    msg = f"{self.name} consume 30 , count = {cnt}"
                    print(msg)
                    
                    con.notify()
                con.release()
                time.sleep(1)
                
                

cnt = 500
con = threading.Condition()


def main():
    for i in range(3):
        p = Producer()
        p.start()
        
    for j in range(20):
        c = Consumer()
        c.start()
        
main()

Thread-16 consume 5 , count = 920
Thread-15 consume 5 , count = 900
Thread-12 consume 5 , count = 880
Thread-13 consume 5 , count = 860
Thread-10 produce 100, count =960
Thread-17 consume 5 , count = 940
Thread-11 consume 5 , count = 920
Thread-14 consume 5 , count = 900
Thread-9 produce 100, count =1000
Thread-18 produce 100, count =600
Thread-19 produce 100, count =700
Thread-20 produce 100, count =800
Thread-21 consume 5 , count = 770
Thread-22 consume 5 , count = 740
Thread-23 consume 5 , count = 710
Thread-24 consume 5 , count = 680
Thread-25 consume 5 , count = 650
Thread-26 consume 5 , count = 620
Thread-27 consume 5 , count = 590
Thread-28 consume 5 , count = 560
Thread-29 consume 5 , count = 530
Thread-30 consume 5 , count = 500
Thread-31 consume 5 , count = 470
Thread-32 consume 5 , count = 440
Thread-33 consume 5 , count = 410
Thread-34 consume 5 , count = 380
Thread-35 consume 5 , count = 350
Thread-36 consume 5 , count = 320
Thread-37 consume 5 , count = 290
Thread-38 cons

Thread-35 consume 5 , count = 70
Thread-8 produce 100, count =170
Thread-37 consume 5 , count = 140
Thread-32 consume 5 , count = 110
Thread-15 consume 5 , count = 90
Thread-10 produce 100, count =190
Thread-13 consume 5 , count = 170
Thread-11 consume 5 , count = 150
Thread-12 consume 5 , count = 130
Thread-30 consume 5 , count = 100
Thread-9 produce 100, count =200
Thread-38 consume 5 , count = 170
Thread-16 consume 5 , count = 150
Thread-29 consume 5 , count = 120
Thread-17 consume 5 , count = 100
Thread-18 produce 100, count =200
Thread-40 consume 5 , count = 170
Thread-19 produce 100, count =270
Thread-14 consume 5 , count = 250
Thread-20 produce 100, count =350
Thread-27 consume 5 , count = 320
Thread-21 consume 5 , count = 290
Thread-26 consume 5 , count = 260
Thread-36 consume 5 , count = 230
Thread-23 consume 5 , count = 200
Thread-24 consume 5 , count = 170
Thread-22 consume 5 , count = 140
Thread-33 consume 5 , count = 110
Thread-25 consume 5 , count = 80
Thread-8 produce 10

Thread-10 produce 100, count =180
Thread-25 consume 5 , count = 150
Thread-27 consume 5 , count = 120
Thread-39 consume 5 , count = 90
Thread-8 produce 100, count =190
Thread-13 consume 5 , count = 170
Thread-11 consume 5 , count = 150
Thread-33 consume 5 , count = 120
Thread-18 produce 100, count =220
Thread-37 consume 5 , count = 190
Thread-31 consume 5 , count = 160
Thread-9 produce 100, count =260
Thread-16 consume 5 , count = 240
Thread-32 consume 5 , count = 210
Thread-30 consume 5 , count = 180
Thread-38 consume 5 , count = 150
Thread-34 consume 5 , count = 120
Thread-12 consume 5 , count = 100
Thread-15 consume 5 , count = 80
Thread-19 produce 100, count =180
Thread-40 consume 5 , count = 150
Thread-29 consume 5 , count = 120
Thread-22 consume 5 , count = 90
Thread-20 produce 100, count =190
Thread-36 consume 5 , count = 160
Thread-23 consume 5 , count = 130
Thread-24 consume 5 , count = 100
Thread-26 consume 5 , count = 70
Thread-10 produce 100, count =170
Thread-14 consume 5 

Thread-10 produce 100, count =240
Thread-26 consume 5 , count = 210
Thread-23 consume 5 , count = 180
Thread-24 consume 5 , count = 150
Thread-17 consume 5 , count = 130
Thread-14 consume 5 , count = 110
Thread-8 produce 100, count =210
Thread-27 consume 5 , count = 180
Thread-25 consume 5 , count = 150
Thread-33 consume 5 , count = 120
Thread-16 consume 5 , count = 100
Thread-11 consume 5 , count = 80
Thread-18 produce 100, count =180
Thread-39 consume 5 , count = 150
Thread-9 produce 100, count =250
Thread-31 consume 5 , count = 220
Thread-34 consume 5 , count = 190
Thread-35 consume 5 , count = 160
Thread-37 consume 5 , count = 130
Thread-32 consume 5 , count = 100
Thread-30 consume 5 , count = 70
Thread-20 produce 100, count =170
Thread-28 consume 5 , count = 140
Thread-40 consume 5 , count = 110
Thread-29 consume 5 , count = 80
Thread-19 produce 100, count =180
Thread-36 consume 5 , count = 150
Thread-10 produce 100, count =250
Thread-26 consume 5 , count = 220
Thread-23 consume 5

Thread-10 produce 100, count =180
Thread-23 consume 5 , count = 150
Thread-26 consume 5 , count = 120
Thread-28 consume 5 , count = 90
Thread-8 produce 100, count =190
Thread-14 consume 5 , count = 170
Thread-18 produce 100, count =270
Thread-12 consume 5 , count = 250
Thread-21 consume 5 , count = 220
Thread-16 consume 5 , count = 200
Thread-29 consume 5 , count = 170
Thread-9 produce 100, count =270
Thread-32 consume 5 , count = 240
Thread-39 consume 5 , count = 210
Thread-11 consume 5 , count = 190
Thread-27 consume 5 , count = 160
Thread-17 consume 5 , count = 140
Thread-25 consume 5 , count = 110
Thread-24 consume 5 , count = 80
Thread-20 produce 100, count =180
Thread-31 consume 5 , count = 150
Thread-34 consume 5 , count = 120
Thread-30 consume 5 , count = 90
Thread-19 produce 100, count =190
Thread-40 consume 5 , count = 160
Thread-38 consume 5 , count = 130
Thread-15 consume 5 , count = 110
Thread-13 consume 5 , count = 90
Thread-10 produce 100, count =190
Thread-28 consume 5 

Thread-10 produce 100, count =170
Thread-13 consume 5 , count = 150
Thread-38 consume 5 , count = 120
Thread-30 consume 5 , count = 90
Thread-8 produce 100, count =190
Thread-21 consume 5 , count = 160
Thread-26 consume 5 , count = 130
Thread-16 consume 5 , count = 110
Thread-23 consume 5 , count = 80
Thread-18 produce 100, count =180
Thread-39 consume 5 , count = 150
Thread-32 consume 5 , count = 120
Thread-29 consume 5 , count = 90
Thread-9 produce 100, count =190
Thread-20 produce 100, count =290
Thread-37 consume 5 , count = 260
Thread-36 consume 5 , count = 230
Thread-25 consume 5 , count = 200
Thread-11 consume 5 , count = 180
Thread-33 consume 5 , count = 150
Thread-12 consume 5 , count = 130
Thread-19 produce 100, count =230
Thread-31 consume 5 , count = 200
Thread-15 consume 5 , count = 180
Thread-24 consume 5 , count = 150
Thread-40 consume 5 , count = 120
Thread-10 produce 100, count =220
Thread-13 consume 5 , count = 200
Thread-38 consume 5 , count = 170
Thread-30 consume 5

In [50]:
### ref: https://www.pythontutorial.net/python-concurrency/python-threading-lock/
### Race condition with multi-thread
### And using lock (mutex) for critical section 
import threading , time 

counter = 0

mutex = threading.Lock()

def increase(by,lock):
    global counter 
    
    
    ### 沒有 lock(mutex), counter 值會亂存取 -> race condition 大炸裂
#     local_count = counter 
#     local_count += by 

#     time.sleep(0.1)

#     counter = local_count
#     print(f"counter={counter}")
    
    
    ### 使用 with 語法, 建構 critical section 
    with lock:
        local_count = counter 
        local_count += by 

        time.sleep(0.1)  ## 造成 race condition 的元兇！？ （發生 thread-level 的 context-switch ）

        counter = local_count
        print(f"counter={counter}")
    
    
    ### 使用 lock.acquire() / lock.release() 打造 critical section !!!
    ### 有 lock (mutex) 者, 才可進入此區執行
#     lock.acquire()
#     local_count = counter 
#     local_count += by 
    
#     time.sleep(0.1)
    
#     counter = local_count
#     print(f"counter={counter}")
#     lock.release()
    

t1 = threading.Thread(target=increase,args=(10,mutex))
t2 = threading.Thread(target=increase,args=(20,mutex))
t3 = threading.Thread(target=increase,args=(30,mutex))

t1.start()
t2.start()
t3.start()


t1.join()
t2.join()
t3.join()

print(f"The final counter is {counter}")

counter=10
counter=30
counter=60
The final counter is 60


In [46]:
### 操作 list 引發 race condition 失敗～～～

# testqq = []

# for i in range(1000):
#     testqq.append(1)
# print(len(testqq))

import threading ,time

testqq = []

def add_element(cnt):
    global testqq 
    
    local_list = testqq
    for i in range(cnt):
        local_list.append(1)
    
    time.sleep(0.1)
    
    testqq = local_list

        
t1 = threading.Thread(target=add_element,args=(300,))
t2 = threading.Thread(target=add_element,args=(180,))

t1.start()
t2.start()

t1.join()
t2.join()


print(f"The final list length: {len(testqq)}")
# print(f"The final list length: {str(total)}")

The final list length: 480
